In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Source: http://home.agh.edu.pl/~kzajac/dydakt/mownit/lab10/pde.pdf

Membrana w kształcie kwadratu o boku a jest równomiernie obciążona.
Membrana znajduje sie w stanie statycznym, a jej brzeg jest sztywno zamocowany i nieodkształcony.
Obliczyc odkształcenie membrany od poziomu , zakładając ze spełnia ono
równanie Poissona:

$$
\frac{\partial^2M}{\partial x^2} + \frac{\partial^2M}{\partial y^2} = \frac{-p}{T}
$$

Gdzie mamy ustalone parametry:
* $p>0$ - ciśnienie wywierane na membrane.
* $T$ - napięcie membrany.

$m$ - wartość napięcia membrany

Oznaczenia:
* $N$ := wymiary siatki
* $d = \frac{a}{N-1}$ := odległość między polami na siatce
* M := macierz $(N) \times (N)$ wartości napięcia membrany w danych punktach

$$
\frac{\partial^2 M}{\partial x^2} = \frac{M_{i+1,j} - 2 \cdot M_{ij} + M_{i-1,j} }{d^2}
$$

$$
\frac{\partial^2 M}{\partial y^2} = \frac{M_{i,j+1} - 2 \cdot M_{ij} + M_{i,j-1} }{d^2}
$$

Zapiszmy układ równań różniczkowych:

$$
\begin{cases}
    \forall i, j \in {1..N} : M_{1,j} = M_{N,j} = M_{i,1} = M_{i,N} = 0 \\
    M_{i+1,j}  + M_{i-1,j} + M_{i,j+1} + M_{i,j-1} - 4 \cdot M_{ij} = \dfrac{-p \cdot d^2}{T}
\end{cases}
$$

W celu zwektoryzowania siatki M, wprowadźmy wektor $\theta$:

$$
M_{ij} = \theta_{N\cdot i + j}
$$

Zauważmy, że dla danego $M{ij} = \theta_k$
$$
\begin{cases}
    M_{i-1, j} = \theta_{k - N} \\
    M_{i+1, j} = \theta_{k + N} \\
    M_{i, j-1} = \theta_{k-1} \\
    M_{i, j+1} = \theta_{k+1}
\end{cases}
$$

Po przekształceniu:

$$
\begin{cases}
    \forall i \in (1 \space .. \space N) : \theta_i = 0 
        \space\space\text{(krawędź górna)}\\
    \forall i \in (1 \space .. \space N) : \theta_{N(N-1) + i} = 0
        \space\space\text{(krawędź dolna)}\\
    \forall j \in (0 \space .. \space N-1) : \theta_{Nj + 1} = 0 
        \space\space\text{(krawędź lewa)}\\
    \forall j \in (1 \space .. \space N) : \theta_{Nj} = 0
        \space\space\text{(krawędź prawa)}\\
    \theta_{k-N}  + \theta_{k+N} + \theta_{k-1} + \theta_{k+1} - 4 \cdot \theta_{k} = \dfrac{-p \cdot d^2}{T}
        \space\space\text{(pozostałe punkty)}\\
\end{cases}
$$


In [ ]:
a = 1.0
p = 1.0
T = 1.0

N = 20
d = a / (N - 1)

a, p, T, N, d
NN = N ** 2 # number of datapoints

In [ ]:
up_edge = np.arange(0,N)
down_edge = np.arange(NN - N, NN)
left_edge = np.arange(N) * N
right_edge = (np.arange(N) + 1) * N - 1

up_edge, down_edge, left_edge, right_edge

Right-hand vector of results

In [ ]:
results = np.array([(-p * d ** 2) / T for _ in range(NN)])
results[up_edge] = results[down_edge] = results[left_edge] = results[right_edge] = 0
results

Matrix of coefficients:

In [ ]:
matrix = np.eye(NN) * -4
matrix[np.arange(1, NN), np.arange(0, NN - 1)] = 1 # elem to the left
matrix[np.arange(0, NN - 1), np.arange(1, NN)] = 1 # elems to the right

matrix[np.arange(N, NN), np.arange(0, NN - N)] = 1 # elems to the up
matrix[np.arange(0, NN - N - 1), np.arange(N, NN - 1)] = 1 # elems to the down

matrix[:, up_edge] = 0
matrix[:, down_edge] = 0
matrix[:, left_edge] = 0
matrix[:, right_edge] = 0

Thetas:

In [ ]:
thetas = np.linalg.lstsq(matrix, results)[0]

In [ ]:
membrane = thetas.reshape(N, N)
membrane[0] = 0
membrane[N-1] = 0
membrane[np.arange(N), 0] = 0
membrane[np.arange(N), N-1] = 0

In [ ]:
sns.heatmap(membrane)